In [1]:
# clear_vision_complete.py

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.autograd import grad
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
import cv2
from tqdm import tqdm
import time
from io import BytesIO
import scipy
from scipy import linalg
import pickle
import json

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#----------------------------------------------------------------------------------
# Dataset and Data Loading Functions
#----------------------------------------------------------------------------------

# Define transformations - using smaller images for CPU processing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Custom dataset class for CelebA
class CelebADataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir)
                          if f.endswith('.jpg') or f.endswith('.png')]
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image

# Load the dataset
def load_dataset(root_dir='/kaggle/input/celebahq-resized-256x256/celeba_hq_256', max_images=None):
    try:
        dataset = CelebADataset(root_dir, transform=transform)
        
        # Limit dataset if specified
        if max_images and max_images < len(dataset):
            indices = list(range(len(dataset)))
            random.shuffle(indices)
            indices = indices[:max_images]
            dataset = torch.utils.data.Subset(dataset, indices)
        
        # Split dataset into train and validation sets
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        
        # Create data loaders - small batch size for CPU
        batch_size = 16
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
        
        print(f"Dataset loaded: {len(dataset)} images")
        print(f"Training set: {len(train_dataset)} images")
        print(f"Validation set: {len(val_dataset)} images")
        
        return train_loader, val_loader
    except Exception as e:
        print(f"Error loading dataset: {e}")
        print("Please ensure the CelebA dataset is correctly placed in the specified directory")
        return None, None

#----------------------------------------------------------------------------------
# Image Corruption Functions
#----------------------------------------------------------------------------------

# 1. Mask-based corruption (as in the original repo)
def apply_mask(images, mask_size=32, offset_x=None, offset_y=None):
    """Apply a square mask to the image at a specified or random position."""
    batch_size, channels, height, width = images.shape
    corrupted_images = images.clone()
    
    # For each image in the batch
    for i in range(batch_size):
        # Determine mask position (random if not specified)
        if offset_x is None:
            offset_x = random.randint(0, width - mask_size)
        if offset_y is None:
            offset_y = random.randint(0, height - mask_size)
        
        # Apply mask (set to 0)
        corrupted_images[i, :, offset_y:offset_y+mask_size, offset_x:offset_x+mask_size] = 0.0
    
    return corrupted_images

# 2. Gaussian noise
def apply_gaussian_noise(images, mean=0.0, std=0.1):
    """Add Gaussian noise to images."""
    noise = torch.randn_like(images) * std + mean
    corrupted_images = images + noise
    # Clip values to be in [-1, 1] range
    corrupted_images = torch.clamp(corrupted_images, -1, 1)
    return corrupted_images

# 3. Salt and pepper noise
def apply_salt_pepper_noise(images, salt_prob=0.02, pepper_prob=0.02):
    """Add salt and pepper noise to images."""
    corrupted_images = images.clone()
    batch_size, channels, height, width = images.shape
    
    # For each image in the batch
    for i in range(batch_size):
        # Generate salt noise (white pixels)
        salt_mask = torch.rand(channels, height, width, device=images.device) < salt_prob
        corrupted_images[i][salt_mask] = 1.0
        
        # Generate pepper noise (black pixels)
        pepper_mask = torch.rand(channels, height, width, device=images.device) < pepper_prob
        corrupted_images[i][pepper_mask] = -1.0
    
    return corrupted_images

# 4. Blur effects
def apply_gaussian_blur(images, kernel_size=7, sigma=1.5):
    """Apply Gaussian blur to images."""
    batch_size = images.shape[0]
    corrupted_images = images.clone()
    
    # Apply Gaussian blur to each image in the batch
    for i in range(batch_size):
        # Make sure kernel size is odd
        if kernel_size % 2 == 0:
            kernel_size += 1
        
        for c in range(3): # Process each channel
            img = images[i, c].cpu().numpy()
            blurred = cv2.GaussianBlur(img, (kernel_size, kernel_size), sigma)
            corrupted_images[i, c] = torch.from_numpy(blurred).to(images.device)
    
    return corrupted_images

# 5. JPEG compression artifacts
def apply_jpeg_artifacts(images, quality=10):
    """Apply JPEG compression artifacts to images."""
    batch_size = images.shape[0]
    corrupted_images = torch.zeros_like(images)
    
    # Apply JPEG compression to each image in the batch
    for i in range(batch_size):
        # Convert to PIL Image
        img = transforms.ToPILImage()((images[i] + 1) / 2) # Denormalize
        
        # Save as JPEG and load back to apply compression artifacts
        buffer = BytesIO()
        img.save(buffer, format="JPEG", quality=quality)
        buffer.seek(0)
        compressed_img = Image.open(buffer)
        
        # Convert back to tensor and normalize
        compressed_tensor = transforms.ToTensor()(compressed_img) * 2 - 1
        corrupted_images[i] = compressed_tensor
    
    return corrupted_images

# 6. Combined corruption (apply multiple types randomly)
def apply_combined_corruption(images):
    """Apply a random combination of corruption types."""
    corrupted_images = images.clone()
    
    # List of corruption functions
    corruptions = [
        lambda x: apply_mask(x, mask_size=random.randint(20, 40)),
        lambda x: apply_gaussian_noise(x, std=random.uniform(0.05, 0.2)),
        lambda x: apply_salt_pepper_noise(x, salt_prob=random.uniform(0.01, 0.05),
                                       pepper_prob=random.uniform(0.01, 0.05)),
        lambda x: apply_gaussian_blur(x, kernel_size=random.choice([3, 5, 7]),
                                    sigma=random.uniform(0.5, 2.0))
    ]
    
    # Apply 1-3 random corruptions
    num_corruptions = random.randint(1, 3)
    selected_corruptions = random.sample(corruptions, num_corruptions)
    
    for corruption_fn in selected_corruptions:
        corrupted_images = corruption_fn(corrupted_images)
    
    return corrupted_images

#----------------------------------------------------------------------------------
# Model Architecture
#----------------------------------------------------------------------------------

class ResidualBlock(nn.Module):
    """Residual block for better gradient flow and feature learning."""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Skip connection if dimensions change
        self.skip = nn.Sequential()
        if in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        residual = x
        out = F.silu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.skip(residual)
        out = F.silu(out)
        return out

class VAE(nn.Module):
    def __init__(self, latent_dim=128):
        super().__init__()
        self.latent_dim = latent_dim
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            
            ResidualBlock(32, 32),
            
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.SiLU(),
            
            ResidualBlock(64, 64),
            
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.SiLU(),
            
            ResidualBlock(128, 128),
            
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.SiLU(),
            
            ResidualBlock(256, 256)
        )
        
        # For 128x128 input, feature map size is now 8x8
        self.encoder_output_size = 8
        encoder_flatten_size = 256 * self.encoder_output_size * self.encoder_output_size
        
        # Latent space mapping
        self.fc_mu = nn.Linear(encoder_flatten_size, latent_dim)
        self.fc_logvar = nn.Linear(encoder_flatten_size, latent_dim)
        self.fc_decoder = nn.Linear(latent_dim, encoder_flatten_size)
        
        # Decoder
        self.decoder = nn.Sequential(
            ResidualBlock(256, 256),
            
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.SiLU(),
            
            ResidualBlock(128, 128),
            
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.SiLU(),
            
            ResidualBlock(64, 64),
            
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            
            ResidualBlock(32, 32),
            
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )
    
    def encode(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1) # Flatten
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z
    
    def decode(self, z):
        x = self.fc_decoder(z)
        x = x.view(x.size(0), 256, self.encoder_output_size, self.encoder_output_size)
        x = self.decoder(x)
        return x
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decode(z)
        return reconstructed, mu, logvar

#----------------------------------------------------------------------------------
# Loss Function and Training Utilities
#----------------------------------------------------------------------------------

# Enhanced VAE loss function
def vae_loss(reconstructed, original, mu, logvar, kld_weight=0.001):
    """
    VAE loss with balanced reconstruction and KL divergence terms.
    Lower KLD weight for better reconstructions.
    """
    # Reconstruction loss (MSE works better for image quality)
    recon_loss = F.mse_loss(reconstructed, original, reduction='sum') / original.size(0)
    
    # KL Divergence
    kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / original.size(0)
    
    # Total loss
    loss = recon_loss + kld_weight * kld_loss
    
    return loss, recon_loss, kld_loss

# Function to save sample images during training
def save_sample_images(model, data_loader, epoch, output_dir='/kaggle/working/samples'):
    """Save sample reconstructions during training."""
    os.makedirs(output_dir, exist_ok=True)
    model.eval()
    
    # Get a batch of images
    batch = next(iter(data_loader))
    if isinstance(batch, list):
        batch = batch[0]  # For ImageFolder dataset
    original_images = batch.to(device)[:8]  # Use 8 images
    
    # Apply different corruptions
    mask_images = apply_mask(original_images.clone())
    gaussian_images = apply_gaussian_noise(original_images.clone())
    salt_pepper_images = apply_salt_pepper_noise(original_images.clone())
    blur_images = apply_gaussian_blur(original_images.clone())
    jpeg_images = apply_jpeg_artifacts(original_images.clone())
    combined_images = apply_combined_corruption(original_images.clone())
    
    # Get reconstructions
    with torch.no_grad():
        mask_reconstructed, _, _ = model(mask_images)
        gaussian_reconstructed, _, _ = model(gaussian_images)
        salt_pepper_reconstructed, _, _ = model(salt_pepper_images)
        blur_reconstructed, _, _ = model(blur_images)
        jpeg_reconstructed, _, _ = model(jpeg_images)
        combined_reconstructed, _, _ = model(combined_images)
    
    # Create comparison grid: original, corrupted, reconstructed
    rows = [
        torch.cat([original_images, mask_images, mask_reconstructed], dim=0),
        torch.cat([original_images, gaussian_images, gaussian_reconstructed], dim=0),
        torch.cat([original_images, salt_pepper_images, salt_pepper_reconstructed], dim=0),
        torch.cat([original_images, blur_images, blur_reconstructed], dim=0),
        torch.cat([original_images, jpeg_images, jpeg_reconstructed], dim=0),
        torch.cat([original_images, combined_images, combined_reconstructed], dim=0)
    ]
    
    # Save grid
    comparison = torch.cat(rows, dim=0)
    save_image(comparison.cpu() * 0.5 + 0.5, f"{output_dir}/epoch_{epoch}.png",
              nrow=8, padding=2, normalize=False)
    
    print(f"Sample images saved for epoch {epoch}")
    model.train()

#----------------------------------------------------------------------------------
# New LPIPS Implementation
#----------------------------------------------------------------------------------

class LPIPSMetric:
    def __init__(self, device='cuda'):
        self.device = device
        # Try to load LPIPS package
        try:
            import lpips
            self.lpips_model = lpips.LPIPS(net='alex').to(device)
            self.has_lpips = True
            print("LPIPS module loaded successfully")
        except ImportError:
            print("Warning: LPIPS package not found. Using AlexNet features as approximation.")
            print("For better results, install LPIPS with: pip install lpips")
            # Use AlexNet features as approximation for LPIPS
            self.has_lpips = False
            self.alexnet = models.alexnet(pretrained=True).features[:9].eval().to(device)
            for param in self.alexnet.parameters():
                param.requires_grad = False
    
    def calculate_lpips(self, img1, img2):
        """Calculate LPIPS between two images"""
        # Ensure images have the same dimensions
        if img1.shape[2:] != img2.shape[2:]:
            img1 = F.interpolate(img1, size=img2.shape[2:], mode='bilinear', align_corners=False)
        
        if self.has_lpips:
            # Use official LPIPS if available
            with torch.no_grad():
                lpips_dist = self.lpips_model(img1, img2)
                return lpips_dist.mean().item()
        else:
            # Use AlexNet feature distance as approximation
            with torch.no_grad():
                # Make sure images are in range [-1, 1]
                if img1.min() < -1 or img1.max() > 1:
                    img1 = torch.clamp(img1, -1, 1)
                if img2.min() < -1 or img2.max() > 1:
                    img2 = torch.clamp(img2, -1, 1)
                
                # Convert from [-1,1] to [0,1] for AlexNet
                img1 = (img1 + 1) / 2
                img2 = (img2 + 1) / 2
                
                # Extract features
                feat1 = self.alexnet(img1)
                feat2 = self.alexnet(img2)
                
                # Calculate distance
                lpips_dist = torch.mean((feat1 - feat2).pow(2))
                return lpips_dist.item()

#----------------------------------------------------------------------------------
# New FID Score Implementation
#----------------------------------------------------------------------------------

class FIDMetric:
    def __init__(self, device='cuda'):
        self.device = device
        
        # Load Inception v3 model for FID computation
        try:
            self.inception_model = models.inception_v3(pretrained=True, transform_input=False)
            self.inception_model.fc = nn.Identity()  # Remove classification layer
            self.inception_model.eval().to(device)
            for param in self.inception_model.parameters():
                param.requires_grad = False
            self.has_inception = True
            print("Inception model loaded for FID calculation")
        except:
            print("Warning: Could not load Inception model. FID scores will not be accurate.")
            try:
                # Try with weights parameter instead
                self.inception_model = models.inception_v3(weights='DEFAULT', transform_input=False)
                self.inception_model.fc = nn.Identity()  # Remove classification layer
                self.inception_model.eval().to(device)
                for param in self.inception_model.parameters():
                    param.requires_grad = False
                self.has_inception = True
                print("Inception model loaded with updated API")
            except:
                self.has_inception = False
    
    def get_activations(self, images, batch_size=50):
        """Get Inception activations for a batch of images"""
        n_batches = len(images) // batch_size + 1
        act = np.empty((len(images), 2048))
        
        for batch_idx in range(n_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, len(images))
            
            if start_idx >= end_idx:
                break
                
            batch = images[start_idx:end_idx].to(self.device)
            
            # Resize to inception input size
            if batch.shape[2] != 299 or batch.shape[3] != 299:
                batch = F.interpolate(batch, size=(299, 299), mode='bilinear', align_corners=False)
            
            # Normalize from [-1, 1] to [0, 1] to [0, 255]
            if batch.min() < 0:
                batch = (batch + 1) / 2  # [-1,1] -> [0,1]
            
            # Forward pass
            with torch.no_grad():
                pred = self.inception_model(batch)
            
            # Store activations
            act[start_idx:end_idx] = pred.cpu().numpy()
        
        return act
    
    def calculate_activation_statistics(self, images):
        """Calculate statistics of activations for FID"""
        activations = self.get_activations(images)
        mu = np.mean(activations, axis=0)
        sigma = np.cov(activations, rowvar=False)
        return mu, sigma
    
    def calculate_frechet_distance(self, mu1, sigma1, mu2, sigma2):
        """Calculate Fréchet distance between two multivariate Gaussians"""
        diff = mu1 - mu2
        
        # Product might be almost singular
        covmean, _ = scipy.linalg.sqrtm(sigma1.dot(sigma2), disp=False)
        if not np.isfinite(covmean).all():
            offset = np.eye(sigma1.shape[0]) * 1e-6
            covmean = scipy.linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))
        
        # Numerical error might give slight imaginary component
        if np.iscomplexobj(covmean):
            covmean = covmean.real
        
        tr_covmean = np.trace(covmean)
        
        return diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean
    
    def calculate_fid(self, real_images, generated_images):
        """Calculate FID between real and generated images"""
        if not self.has_inception:
            return -1  # Cannot calculate FID without inception model
        
        # Get statistics for real and generated images
        mu_real, sigma_real = self.calculate_activation_statistics(real_images)
        mu_gen, sigma_gen = self.calculate_activation_statistics(generated_images)
        
        # Calculate FID
        fid_score = self.calculate_frechet_distance(mu_real, sigma_real, mu_gen, sigma_gen)
        return fid_score

#----------------------------------------------------------------------------------
# Evaluation Metrics
#----------------------------------------------------------------------------------

# Basic evaluation metrics
def compute_psnr(img1, img2):
    """Compute Peak Signal-to-Noise Ratio between image tensors."""
    # Convert to range [0, 1] if in [-1, 1]
    if img1.min() < 0:
        img1 = img1 * 0.5 + 0.5
    if img2.min() < 0:
        img2 = img2 * 0.5 + 0.5
    
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

def compute_ssim(img1, img2):
    """Compute Structural Similarity Index between image tensors."""
    # Convert to range [0, 1] if in [-1, 1]
    if img1.min() < 0:
        img1 = img1 * 0.5 + 0.5
    if img2.min() < 0:
        img2 = img2 * 0.5 + 0.5
    
    # Constants
    C1 = (0.01 * 1) ** 2
    C2 = (0.03 * 1) ** 2
    
    # Calculate mean, variance, covariance
    mu1 = torch.mean(img1, dim=(2, 3), keepdim=True)
    mu2 = torch.mean(img2, dim=(2, 3), keepdim=True)
    
    sigma1_sq = torch.mean((img1 - mu1) ** 2, dim=(2, 3), keepdim=True)
    sigma2_sq = torch.mean((img2 - mu2) ** 2, dim=(2, 3), keepdim=True)
    sigma12 = torch.mean((img1 - mu1) * (img2 - mu2), dim=(2, 3), keepdim=True)
    
    # SSIM formula
    numerator = (2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)
    denominator = (mu1**2 + mu2**2 + C1) * (sigma1_sq + sigma2_sq + C2)
    ssim = torch.mean(numerator / denominator)
    
    return ssim.item()

# Comprehensive evaluation with all metrics
def evaluate_model(model, data_loader, corruption_types=['mask', 'gaussian', 'salt_pepper', 'blur', 'jpeg', 'combined']):
    """Evaluate model on multiple corruption types with PSNR, SSIM, LPIPS and FID."""
    model.eval()
    results = {}
    
    # Initialize metrics
    lpips_metric = LPIPSMetric(device)
    fid_metric = FIDMetric(device)
    
    # Get a batch of images
    batch = next(iter(data_loader))
    if isinstance(batch, list):
        batch = batch[0]
    original_images = batch.to(device)[:32]  # Use 32 images for evaluation
    
    corruption_functions = {
        'mask': apply_mask,
        'gaussian': apply_gaussian_noise,
        'salt_pepper': apply_salt_pepper_noise,
        'blur': apply_gaussian_blur,
        'jpeg': apply_jpeg_artifacts,
        'combined': apply_combined_corruption
    }
    
    # Create progress bar for corruption types evaluation
    corruption_bar = tqdm(corruption_types, desc="Evaluating corruption types")
    
    for corruption_type in corruption_bar:
        corruption_bar.set_description(f"Evaluating {corruption_type}")
        
        if corruption_type not in corruption_functions:
            print(f"Unknown corruption type: {corruption_type}")
            continue
        
        # Apply corruption
        corruption_fn = corruption_functions[corruption_type]
        corrupted_images = corruption_fn(original_images.clone())
        
        # Get reconstructions
        with torch.no_grad():
            start_time = time.time()
            reconstructed_images, _, _ = model(corrupted_images)
            inference_time = time.time() - start_time
        
        # Calculate metrics - silently, without progress bar
        avg_psnr = 0
        avg_ssim = 0
        lpips_scores = []
        
        # Use regular loop instead of tqdm progress bar to avoid per-percentage updates
        for i in range(len(original_images)):
            psnr = compute_psnr(original_images[i:i+1], reconstructed_images[i:i+1])
            ssim = compute_ssim(original_images[i:i+1], reconstructed_images[i:i+1])
            lpips = lpips_metric.calculate_lpips(original_images[i:i+1], reconstructed_images[i:i+1])
            
            avg_psnr += psnr
            avg_ssim += ssim
            lpips_scores.append(lpips)
        
        avg_psnr /= len(original_images)
        avg_ssim /= len(original_images)
        avg_lpips = sum(lpips_scores) / len(lpips_scores)
        
        # Calculate FID (this is computationally expensive)
        try:
            fid_score = fid_metric.calculate_fid(original_images, reconstructed_images)
        except:
            fid_score = -1
            print(f"Warning: Failed to calculate FID score for {corruption_type}")
        
        results[corruption_type] = {
            'psnr': avg_psnr,
            'ssim': avg_ssim,
            'lpips': avg_lpips,
            'fid': fid_score,
            'inference_time_ms': (inference_time / len(original_images)) * 1000
        }
        
        corruption_bar.set_postfix(psnr=f"{avg_psnr:.2f}dB", ssim=f"{avg_ssim:.4f}", lpips=f"{avg_lpips:.4f}")
    
    model.train()
    return results


#----------------------------------------------------------------------------------
# Training Function
#----------------------------------------------------------------------------------
def train_vae(model, train_loader, val_loader, num_epochs=100, lr=1e-4, kld_weight=0.001, 
              save_dir='/kaggle/working/checkpoints'):
    """Train the VAE model with multiple corruption types."""
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs('/kaggle/working/samples', exist_ok=True)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    # Track learning rate changes manually
    previous_lr = optimizer.param_groups[0]['lr']
    
    # Lists to store loss values
    train_losses = []
    val_losses = []
    
    # Use different corruption types during training
    corruption_types = ['mask', 'gaussian', 'salt_pepper', 'blur', 'jpeg', 'combined']
    corruption_functions = {
        'mask': apply_mask,
        'gaussian': apply_gaussian_noise,
        'salt_pepper': apply_salt_pepper_noise,
        'blur': apply_gaussian_blur,
        'jpeg': apply_jpeg_artifacts,
        'combined': apply_combined_corruption
    }
    
    print("Starting training...")
    
    # Create overall progress bar for epochs
    try:
        epoch_bar = tqdm(range(num_epochs), desc="Training Progress", position=0)
        
        for epoch in epoch_bar:
            model.train()
            epoch_train_loss = 0
            epoch_recon_loss = 0
            epoch_kld_loss = 0
            batch_count = 0
            
            # Training loop - without detailed per-batch output
            for batch in train_loader:
                # Move batch to device
                if isinstance(batch, list):
                    batch = batch[0]
                original = batch.to(device)
                
                # Apply random corruption
                corruption_type = random.choice(corruption_types)
                corrupted = corruption_functions[corruption_type](original.clone())
                
                # Forward pass
                reconstructed, mu, logvar = model(corrupted)
                
                # Compute loss
                loss, recon_loss, kld_loss = vae_loss(reconstructed, original, mu, logvar, kld_weight)
                
                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                # Accumulate losses without printing
                epoch_train_loss += loss.item()
                epoch_recon_loss += recon_loss.item()
                epoch_kld_loss += kld_loss.item()
                batch_count += 1
            
            # Average training loss for this epoch
            avg_train_loss = epoch_train_loss / batch_count
            avg_recon_loss = epoch_recon_loss / batch_count
            avg_kld_loss = epoch_kld_loss / batch_count
            train_losses.append(avg_train_loss)
            
            # Validation - without detailed per-batch output
            model.eval()
            epoch_val_loss = 0
            val_batch_count = 0
            
            with torch.no_grad():
                for batch in val_loader:
                    if isinstance(batch, list):
                        batch = batch[0]
                    original = batch.to(device)
                    
                    # Apply random corruption for validation
                    corruption_type = random.choice(corruption_types)
                    corrupted = corruption_functions[corruption_type](original.clone())
                    
                    # Forward pass
                    reconstructed, mu, logvar = model(corrupted)
                    
                    # Compute loss
                    loss, _, _ = vae_loss(reconstructed, original, mu, logvar, kld_weight)
                    epoch_val_loss += loss.item()
                    val_batch_count += 1
            
            # Average validation loss for this epoch
            avg_val_loss = epoch_val_loss / val_batch_count
            val_losses.append(avg_val_loss)
            
            # Update the main epoch progress bar
            epoch_bar.set_postfix(train_loss=f"{avg_train_loss:.4f}", val_loss=f"{avg_val_loss:.4f}")
            
            # Print epoch summary - ONLY PRINT ONCE PER EPOCH
            print(f"\nEpoch {epoch+1}/{num_epochs}:")
            print(f" Train Loss: {avg_train_loss:.6f} (Recon: {avg_recon_loss:.6f}, KLD: {avg_kld_loss:.6f})")
            print(f" Validation Loss: {avg_val_loss:.6f}")
            print(f" Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
            
            # Save model checkpoint
            if (epoch + 1) % 10 == 0 or epoch == num_epochs - 1:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_loss': avg_train_loss,
                    'val_loss': avg_val_loss,
                    'latent_dim': model.latent_dim
                }, f"{save_dir}/vae_epoch_{epoch+1}.pth")
                print(f"Model checkpoint saved to {save_dir}/vae_epoch_{epoch+1}.pth")
            
            # Generate and save sample images
            if (epoch + 1) % 10 == 0 or epoch == num_epochs - 1:
                save_sample_images(model, val_loader, epoch + 1)
            
            # Evaluate model on validation set (every 5 epochs or at the end)
            if (epoch + 1) % 10 == 0 or epoch == num_epochs - 1:
                print(f"\nRunning evaluation at epoch {epoch+1}...")
                eval_results = evaluate_model(model, val_loader)
                print("\nValidation Metrics:")
                for corruption_type, metrics in eval_results.items():
                    print(f" {corruption_type}: PSNR = {metrics['psnr']:.2f}dB, SSIM = {metrics['ssim']:.4f}, " +
                        f"LPIPS = {metrics['lpips']:.4f}, FID = {metrics['fid']:.2f}, " +
                        f"Inference Time = {metrics['inference_time_ms']:.2f}ms")
                
                # Save metrics to file for later analysis
                metrics_file = f"{save_dir}/metrics_epoch_{epoch+1}.json"
                with open(metrics_file, 'w') as f:
                    json.dump(eval_results, f, indent=2)
            
            # Update learning rate
            scheduler.step(avg_val_loss)
            
            # Check for learning rate changes and log them
            current_lr = optimizer.param_groups[0]['lr']
            if current_lr != previous_lr:
                print(f"\nLearning rate adjusted from {previous_lr:.6f} to {current_lr:.6f}")
                previous_lr = current_lr
            
    except KeyboardInterrupt:
        print("\nTraining interrupted by user.")
    except Exception as e:
        print(f"\nError during training: {e}")
    
    # Plot training and validation loss
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig('/kaggle/working/loss_curve.png')
    plt.close()
    
    print("Training completed!")
    
    # Save the final model in Kaggle output directory
    final_model_path = '/kaggle/working/vae_final_model.pth'
    torch.save({
        'epoch': num_epochs,
        'model_state_dict': model.state_dict(),
        'latent_dim': model.latent_dim
    }, final_model_path)
    print(f"Final model saved to {final_model_path}")
    
    return train_losses, val_losses
 
            
         
    
   
    
    

#----------------------------------------------------------------------------------
# Image Restoration and Visualization
#----------------------------------------------------------------------------------

def restore_image(model, image_path, corruption_type='combined', save_path=None):
    """Restore a single image using the trained VAE model."""
    if isinstance(model, dict):
        # If model is a state dict, create a new model instance
        latent_dim = model.get('latent_dim', 128)
        model_instance = VAE(latent_dim=latent_dim).to(device)
        model_instance.load_state_dict(model['model_state_dict'])
        model = model_instance
        
    model.eval()
    
    # Load and preprocess image
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    image = Image.open(image_path).convert('RGB')
    original = transform(image).unsqueeze(0).to(device)
    
    # Apply corruption
    corruption_functions = {
        'mask': apply_mask,
        'gaussian': apply_gaussian_noise,
        'salt_pepper': apply_salt_pepper_noise,
        'blur': apply_gaussian_blur,
        'jpeg': apply_jpeg_artifacts,
        'combined': apply_combined_corruption
    }
    
    if corruption_type not in corruption_functions:
        print(f"Unknown corruption type: {corruption_type}. Using 'combined' instead.")
        corruption_type = 'combined'
    
    corrupted = corruption_functions[corruption_type](original.clone())
    
    # Restore image
    with torch.no_grad():
        restored, _, _ = model(corrupted)
    
    # Convert tensors to images for visualization
    # Convert from [-1, 1] range to [0, 1]
    original_img = original.cpu() * 0.5 + 0.5
    corrupted_img = corrupted.cpu() * 0.5 + 0.5
    restored_img = restored.cpu() * 0.5 + 0.5
    
    # Create comparison grid
    comparison = torch.cat([original_img, corrupted_img, restored_img], dim=0)
    
    # Save or display results
    if save_path:
        save_image(comparison, save_path, nrow=1, padding=5)
        print(f"Restored image saved to {save_path}")
    
    # Calculate metrics
    psnr = compute_psnr(original, restored)
    ssim = compute_ssim(original, restored)
    
    # Initialize LPIPS
    lpips_metric = LPIPSMetric(device)
    lpips_val = lpips_metric.calculate_lpips(original, restored)
    
    print(f"Restoration Metrics:")
    print(f" PSNR: {psnr:.2f}dB")
    print(f" SSIM: {ssim:.4f}")
    print(f" LPIPS: {lpips_val:.4f}")
    
    # For displaying in notebook
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(original_img.squeeze().permute(1, 2, 0).numpy())
    plt.title("Original")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(corrupted_img.squeeze().permute(1, 2, 0).numpy())
    plt.title(f"Corrupted ({corruption_type})")
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(restored_img.squeeze().permute(1, 2, 0).numpy())
    plt.title(f"Restored (PSNR: {psnr:.2f}dB)")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return original, corrupted, restored, {'psnr': psnr, 'ssim': ssim, 'lpips': lpips_val}

#----------------------------------------------------------------------------------
# Loading Pretrained Model
#----------------------------------------------------------------------------------

def load_pretrained_model(checkpoint_path):
    """Load a pretrained VAE model."""
    if not os.path.exists(checkpoint_path):
        print(f"Checkpoint not found: {checkpoint_path}")
        return None
    
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    # Get latent dimension from checkpoint or use default
    latent_dim = checkpoint.get('latent_dim', 128)
    
    # Initialize model
    model = VAE(latent_dim=latent_dim).to(device)
    
    # Load model state
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        # Assume it is a direct state dict
        model.load_state_dict(checkpoint)
    
    print(f"Loaded pretrained model from {checkpoint_path}")
    if 'epoch' in checkpoint:
        print(f"Trained for {checkpoint['epoch']} epochs")
    
    return model

#----------------------------------------------------------------------------------
# Main Pipeline
#------------------------------------------------------------------------------


Using device: cuda


In [2]:
def main(max_images=30000, num_epochs=100):
    """Run the complete pipeline: load data, train model, evaluate and test."""
    print("Starting ClearVision Pipeline...")
    
    # Set paths for Kaggle
    data_dir = '/kaggle/input/celebahq-resized-256x256/celeba_hq_256'
    output_dir = '/kaggle/working'
    
    # Create output directories
    os.makedirs(f'{output_dir}/checkpoints', exist_ok=True)
    os.makedirs(f'{output_dir}/samples', exist_ok=True)
    os.makedirs(f'{output_dir}/metrics', exist_ok=True)
    
    # Create a progress bar for the main pipeline steps
    pipeline_steps = ["Loading datasets", "Model initialization", "Training", "Evaluation", "Saving model"]
    pipeline_bar = tqdm(pipeline_steps, desc="ClearVision Pipeline")
    
    # Step 1: Load datasets with limit
    pipeline_bar.set_description("Loading datasets...")
    train_loader, val_loader = load_dataset(data_dir, max_images=max_images)
    
    if train_loader is None or val_loader is None:
        print("Failed to load datasets. Please check the dataset paths.")
        return
    pipeline_bar.update(1)
    
    # Step 2: Initialize model
    pipeline_bar.set_description("Initializing VAE model...")
    latent_dim = 128
    model = VAE(latent_dim=latent_dim).to(device)
    print(f"Model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
    pipeline_bar.update(1)
    
    # Step 3: Train model
    pipeline_bar.set_description("Training model...")
    train_losses, val_losses = train_vae(model, train_loader, val_loader, 
                                       num_epochs=num_epochs, 
                                       lr=1e-4, 
                                       kld_weight=0.001,
                                       save_dir=f'{output_dir}/checkpoints')
    pipeline_bar.update(1)
    
    # Step 4: Final evaluation on validation set
    pipeline_bar.set_description("Evaluating final model...")
    eval_results = evaluate_model(model, val_loader)
    print("\nFinal Metrics:")
    for corruption_type, metrics in eval_results.items():
        print(f" {corruption_type}: PSNR = {metrics['psnr']:.2f}dB, SSIM = {metrics['ssim']:.4f}, LPIPS = {metrics.get('lpips', 'N/A')}")
        if 'inference_time_ms' in metrics:
            print(f" {corruption_type} Inference Time: {metrics['inference_time_ms']:.2f}ms")
    
    # Save metrics to file
    metrics_file = os.path.join(output_dir, 'metrics', 'final_metrics.json')
    with open(metrics_file, 'w') as f:
        import json
        # Convert values that might not be JSON serializable
        serializable_results = {}
        for k, v in eval_results.items():
            serializable_results[k] = {k2: float(v2) if not isinstance(v2, str) else v2 
                                     for k2, v2 in v.items()}
        json.dump(serializable_results, f, indent=4)
    
    print(f"Metrics saved to {metrics_file}")
    pipeline_bar.update(1)
    
    # Step 5: Save the final model
    pipeline_bar.set_description("Saving final model...")
    final_model_path = os.path.join(output_dir, 'checkpoints', 'vae_final_model.pth')
    torch.save({
        'epoch': num_epochs,
        'model_state_dict': model.state_dict(),
        'latent_dim': latent_dim
    }, final_model_path)
    print(f"Final model saved to {final_model_path}")
    pipeline_bar.update(1)
    
    pipeline_bar.close()
    print("\nClearVision Pipeline completed!")
    
    # Create a sample image with all corruption types for the final model
    create_final_comparison_image(model, val_loader, 
                                output_path=os.path.join(output_dir, 'final_results.png'))
    
    return model, train_loader, val_loader

def create_final_comparison_image(model, val_loader, output_path):
    """Create a final comparison image with all corruption types"""
    model.eval()
    
    # Get a batch of images
    batch = next(iter(val_loader))
    if isinstance(batch, list):
        batch = batch[0]
    original_image = batch[0:1].to(device)  # Use first image
    
    corruption_types = ['mask', 'gaussian', 'salt_pepper', 'blur', 'jpeg', 'combined']
    corruption_functions = {
        'mask': apply_mask,
        'gaussian': apply_gaussian_noise,
        'salt_pepper': apply_salt_pepper_noise,
        'blur': apply_gaussian_blur,
        'jpeg': apply_jpeg_artifacts,
        'combined': apply_combined_corruption
    }
    
    # Create rows for each corruption type
    rows = []
    
    for corruption_type in corruption_types:
        # Apply corruption
        corruption_fn = corruption_functions[corruption_type]
        corrupted = corruption_fn(original_image.clone())
        
        # Generate restored image
        with torch.no_grad():
            restored, _, _ = model(corrupted)
        
        # Add to rows: original, corrupted, restored
        row = torch.cat([original_image, corrupted, restored], dim=0)
        rows.append(row)
    
    # Create final grid
    grid = torch.cat(rows, dim=0)
    save_image(grid * 0.5 + 0.5, output_path, nrow=3)
    print(f"Final comparison image saved to {output_path}")


In [3]:
def test_model(model, val_loader, num_samples=5, output_dir='/kaggle/working/test_outputs'):
    """Test the trained model on the validation set and generate example outputs"""
    os.makedirs(output_dir, exist_ok=True)
    model.eval()
    
    # Get corruption types
    corruption_types = ['mask', 'gaussian', 'salt_pepper', 'blur', 'jpeg', 'combined']
    corruption_functions = {
        'mask': apply_mask,
        'gaussian': apply_gaussian_noise,
        'salt_pepper': apply_salt_pepper_noise,
        'blur': apply_gaussian_blur,
        'jpeg': apply_jpeg_artifacts,
        'combined': apply_combined_corruption
    }
    
    # Evaluate and report metrics for each corruption type
    print("\nModel Testing Results:")
    
    for corruption_type in corruption_types:
        # Get a batch of samples
        batch = next(iter(val_loader))
        if isinstance(batch, list):
            batch = batch[0]
        
        # Select random samples
        indices = torch.randperm(len(batch))[:num_samples]
        original_images = batch[indices].to(device)
        
        # Apply corruption
        corruption_fn = corruption_functions[corruption_type]
        corrupted_images = corruption_fn(original_images)
        
        # Generate restored images
        with torch.no_grad():
            restored_images, _, _ = model(corrupted_images)
        
        # Calculate metrics
        avg_psnr = 0
        avg_ssim = 0
        lpips_scores = []
        
        # Initialize LPIPS metric
        lpips_metric = LPIPSMetric(device)
        
        for i in range(len(original_images)):
            psnr = compute_psnr(original_images[i:i+1], restored_images[i:i+1])
            ssim = compute_ssim(original_images[i:i+1], restored_images[i:i+1])
            lpips = lpips_metric.calculate_lpips(original_images[i:i+1], restored_images[i:i+1])
            
            avg_psnr += psnr
            avg_ssim += ssim
            lpips_scores.append(lpips)
        
        avg_psnr /= len(original_images)
        avg_ssim /= len(original_images)
        avg_lpips = sum(lpips_scores) / len(lpips_scores)
        
        print(f"{corruption_type.capitalize()} Corruption:")
        print(f"  PSNR: {avg_psnr:.2f}dB")
        print(f"  SSIM: {avg_ssim:.4f}")
        print(f"  LPIPS: {avg_lpips:.4f}")
        
        # Save example images
        for i in range(len(original_images)):
            comparison = torch.cat([
                original_images[i:i+1], 
                corrupted_images[i:i+1], 
                restored_images[i:i+1]
            ], dim=0)
            
            save_image(
                comparison * 0.5 + 0.5,
                f"{output_dir}/{corruption_type}_sample_{i+1}.png",
                nrow=3, 
                padding=5
            )
    
    # Create a comprehensive test grid with all corruption types
    create_test_grid(model, val_loader, output_dir)
    
    print(f"\nTest images saved to {output_dir}")
    return

def create_test_grid(model, val_loader, output_dir):
    """Create a comprehensive test grid with all corruption types"""
    model.eval()
    
    # Get a batch of images
    batch = next(iter(val_loader))
    if isinstance(batch, list):
        batch = batch[0]
    
    # Use 2 sample images
    original_images = batch[:2].to(device)
    
    corruption_types = ['mask', 'gaussian', 'salt_pepper', 'blur', 'jpeg', 'combined']
    corruption_functions = {
        'mask': apply_mask,
        'gaussian': apply_gaussian_noise,
        'salt_pepper': apply_salt_pepper_noise,
        'blur': apply_gaussian_blur,
        'jpeg': apply_jpeg_artifacts,
        'combined': apply_combined_corruption
    }
    
    # Process each image
    all_rows = []
    for img_idx in range(len(original_images)):
        image_rows = []
        original_image = original_images[img_idx:img_idx+1]
        
        for corruption_type in corruption_types:
            corruption_fn = corruption_functions[corruption_type]
            corrupted = corruption_fn(original_image.clone())
            
            with torch.no_grad():
                restored, _, _ = model(corrupted)
            
            # Create row: original, corrupted, restored
            image_rows.append(torch.cat([original_image, corrupted, restored], dim=0))
        
        # Combine all corruption types for this image
        all_rows.append(torch.cat(image_rows, dim=0))
    
    # Create final grid
    grid = torch.cat(all_rows, dim=0)
    save_image(grid * 0.5 + 0.5, f"{output_dir}/comprehensive_test.png", nrow=3)


In [4]:
if __name__ == '__main__':
    # Create necessary directories
    os.makedirs('checkpoints', exist_ok=True)
    os.makedirs('samples', exist_ok=True)
    os.makedirs('outputs', exist_ok=True)
    
    # Run the main pipeline
    model, train_loader, val_loader = main()
    
    # Test the model
    test_model(model, val_loader)
    
    print("""
    ## ClearVision Image Restoration Complete
    
    The model has been trained to restore corrupted images using a VAE architecture.
    It can handle multiple corruption types:
    - Masked regions (inpainting)
    - Gaussian noise
    - Salt and pepper noise
    - Blur
    - JPEG compression artifacts
    - Combined corruptions
    
    To test on your own images, use:
    ```
    restore_image(model, 'path/to/your/image.jpg', corruption_type='combined')
    ```
    
    Available corruption types: 'mask', 'gaussian', 'salt_pepper', 'blur', 'jpeg', 'combined'
    """)


Starting ClearVision Pipeline...


Initializing VAE model...:  20%|██        | 1/5 [00:00<00:01,  2.26it/s]

Dataset loaded: 30000 images
Training set: 27000 images
Validation set: 3000 images


Training model...:  40%|████      | 2/5 [00:00<00:01,  2.88it/s]        

Model parameters: 10,828,739
Starting training...


...

Evaluating final model...:  60%|██████    | 3/5 [1:51:48<1:42:05, 3062.96s/it]

Final model saved to /kaggle/working/vae_final_model.pth
For better results, install LPIPS with: pip install lpips
Inception model loaded for FID calculation



Saving final model...: 100%|██████████| 5/5 [1:52:30<00:00, 1350.13s/it]


Final Metrics:
 mask: PSNR = 24.07dB, SSIM = 0.9617, LPIPS = 1.4006945230066776
 mask Inference Time: 0.36ms
 gaussian: PSNR = 24.45dB, SSIM = 0.9649, LPIPS = 1.34086924046278
 gaussian Inference Time: 0.28ms
 salt_pepper: PSNR = 24.40dB, SSIM = 0.9639, LPIPS = 1.385096449404955
 salt_pepper Inference Time: 0.26ms
 blur: PSNR = 24.33dB, SSIM = 0.9631, LPIPS = 1.576094999909401
 blur Inference Time: 0.25ms
 jpeg: PSNR = 24.30dB, SSIM = 0.9638, LPIPS = 1.3601593896746635
 jpeg Inference Time: 0.25ms
 combined: PSNR = 24.21dB, SSIM = 0.9617, LPIPS = 1.5392706990242004
 combined Inference Time: 0.24ms
Metrics saved to /kaggle/working/metrics/final_metrics.json
Final model saved to /kaggle/working/checkpoints/vae_final_model.pth

ClearVision Pipeline completed!


Final comparison image saved to /kaggle/working/final_results.png

Model Testing Results:
For better results, install LPIPS with: pip install lpips
Mask Corruption:
  PSNR: 24.13dB
  SSIM: 0.9735
  LPIPS: 1.4343
For better results, install LPIPS with: pip install lpips
Gaussian Corruption:
  PSNR: 24.42dB
  SSIM: 0.9676
  LPIPS: 1.2685
For better results, install LPIPS with: pip install lpips
Salt_pepper Corruption:
  PSNR: 24.63dB
  SSIM: 0.9596
  LPIPS: 1.6075
For better results, install LPIPS with: pip install lpips
Blur Corruption:
  PSNR: 24.33dB
  SSIM: 0.9628
  LPIPS: 2.1067
For better results, install LPIPS with: pip install lpips
Jpeg Corruption:
  PSNR: 24.03dB
  SSIM: 0.9590
  LPIPS: 1.4422
For better results, install LPIPS with: pip install lpips
Combined Corruption:
  PSNR: 22.78dB
  SSIM: 0.9537
  LPIPS: 1.9756

Test images saved to /kaggle/working/test_outputs

    ## ClearVision Image Restoration Complete
    
    The model has been trained to restore corrupted images u